In [2]:
# Imports
from tensorflow.keras.layers import TextVectorization
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers


In [3]:
# Import data
data = pd.read_csv("../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]

train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [4]:
vectoriser = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectoriser.adapt(text_ds)

In [5]:
voc = vectoriser.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [6]:
glove_path = "glove.6B.100d.txt"
embeddings_index = {}
with open(glove_path,encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,"f",sep= " ")
        embeddings_index[word] = coefs

In [7]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens,embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

In [8]:
embedding_layer = Embedding(num_tokens,embedding_dim,embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False)

In [9]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(units=1)(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         1383900   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         12928     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         16512 

In [10]:
x_train = vectoriser(np.array([[s] for s in train_samples])).numpy()
x_val = vectoriser(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(test_labels)

In [11]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError(),tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [12]:
history = model.fit(x=x_train,y=y_train,batch_size=32,epochs=100)

Epoch 1/100
176/176 [==============================] - 4s 19ms/step - loss: 3.4668 - mean_squared_error: 3.4668 - mean_absolute_error: 1.4414 - mean_absolute_percentage_error: 36.7682
Epoch 2/100
176/176 [==============================] - 4s 21ms/step - loss: 1.2474 - mean_squared_error: 1.2474 - mean_absolute_error: 0.8984 - mean_absolute_percentage_error: 24.6271
Epoch 3/100
176/176 [==============================] - 5s 29ms/step - loss: 1.1131 - mean_squared_error: 1.1131 - mean_absolute_error: 0.8416 - mean_absolute_percentage_error: 22.9285
Epoch 4/100
176/176 [==============================] - 5s 30ms/step - loss: 1.0360 - mean_squared_error: 1.0360 - mean_absolute_error: 0.8139 - mean_absolute_percentage_error: 22.1445
Epoch 5/100
176/176 [==============================] - 5s 29ms/step - loss: 0.9997 - mean_squared_error: 0.9997 - mean_absolute_error: 0.7997 - mean_absolute_percentage_error: 21.6859
Epoch 6/100
176/176 [==============================] - 5s 28ms/step - loss: 0.92

In [13]:
mse = history.history['mean_squared_error']
mae = history.history['mean_absolute_error']
mpe = history.history['mean_absolute_percentage_error']

scores = pd.DataFrame()
scores["MSE"] = mse
scores["MAE"] = mae
scores["% Error"] = mpe

scores

,MSE,MAE,% Error
0,3.466813,1.441424,36.768200
1,1.247407,0.898405,24.627058
2,1.113067,0.841636,22.928469
3,1.036033,0.813882,22.144453
4,0.999670,0.799659,21.685909
...,...,...,...
95,0.305326,0.432868,11.090082
96,0.288539,0.420973,10.865730
97,0.288064,0.423097,10.979000
98,0.294685,0.425035,10.854995


In [30]:
scores = model.evaluate(x=x_val,y=y_val)
scores[1:]

44/44 [==============================] - 0s 5ms/step - loss: 0.8018 - mean_squared_error: 0.8018 - mean_absolute_error: 0.7049 - mean_absolute_percentage_error: 19.4670


[0.801812469959259, 0.7049391269683838, 19.467012405395508]